# Text tokenization, word indexing, word vectors

## Using keras to tokenize

In [79]:
import numpy as np
from keras.preprocessing.text import Tokenizer

In [80]:
# sample tweets from my twitter inbox with added text for experimentation
samples = [
    """Tesla Motors has nothing to do with this tweet.
    On those rare occasions when I really, really need to reduce the
    size of a file I use "xz -9". Today I found out about the "extreme" setting
    and "xz -e9" squeezed files down another 15% or so. It is not exactly quick,
    but that doesn't really matter in such cases!""",
    
    """Securities and exchange commission has nothing to do with this tweet.
    Do grad students get paid a lot? No. But do we at least have solid
    job security? Also, no. But are we at least ensured a stress-free work
    environment with a healthy work-life balance? Still, also no.""",

    """A design process hyperfocused on A/B testing can result in dark patterns even
    if that’s not the intent. That’s because most A/B tests are based on metrics
    that are relevant to the company’s bottom line, even if they result in harm to users."""
]

tokenizer = Tokenizer(num_words=30)
tokenizer.fit_on_texts(samples)

In [81]:
sequences = tokenizer.texts_to_sequences(samples)
np.array(sequences[0]) # token sequence for first sample

array([13, 14,  2,  3,  5, 15, 16,  6,  7,  8,  8,  2,  4,  1,  7, 17,  7,
        4, 18, 17, 19,  9, 20,  8, 10])

In [82]:
# one-hot for samples
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')
one_hot_results

array([[0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
        1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1.]])

In [83]:
# count in one-hot positions for samples
one_hot_results = tokenizer.texts_to_matrix(samples, mode='count')
one_hot_results

array([[0., 1., 2., 1., 2., 1., 1., 3., 3., 1., 1., 0., 0., 1., 1., 1.,
        1., 2., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 3., 1., 3., 0., 2., 0., 0., 0., 2., 0., 3., 1., 1., 1., 1.,
        1., 0., 1., 0., 0., 2., 2., 2., 2., 2., 0., 0., 0., 0.],
       [0., 3., 2., 0., 2., 0., 2., 0., 0., 0., 2., 0., 2., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 2., 2., 2., 2.]])

## "One-hot hash trick"

Instead of using unique word index into a dictionary of unique words from all samples, compute hash function on word as its code. Lets us limit size of word vector to fixed length rather than length of vocab or whatever. 

In [100]:
def hash(w):
    h = 0
    for c in w:
        h = (h<<3) + ord(c)
    return h

def hashwords(dimensionality = 1000):
    collisions = set()
    codes = set()
    index = {}
    for w in words:
        wcode = hash(w)%dimensionality
        if wcode in codes:
            collisions.add(wcode)
        codes.add(wcode)
        index[w] = wcode
    return index, collisions

words = tokenizer.word_index.keys() # get tokenized words
dimensionality=10
index, collisions = hashwords(dimensionality)
print("INDEX:", index)
print(f"{len(collisions)}/{len(words)}={(len(collisions)*100)/len(words):.1f}% collisions using {dimensionality} unique codes")

INDEX: {'a': 7, 'to': 9, 'do': 1, 'the': 7, 'with': 0, 'on': 8, 'i': 5, 'really': 9, 'but': 4, 'in': 0, 'no': 1, 'are': 1, 'has': 7, 'nothing': 5, 'this': 3, 'tweet': 2, 'xz': 2, 'and': 8, 'not': 4, 'that': 0, 'we': 3, 'at': 2, 'least': 4, 'also': 7, 'work': 1, 'b': 8, 'result': 6, 'even': 2, 'if': 2, 'that’s': 1, 'tesla': 9, 'motors': 1, 'those': 9, 'rare': 9, 'occasions': 5, 'when': 2, 'need': 2, 'reduce': 9, 'size': 7, 'of': 0, 'file': 9, 'use': 9, '9': 7, 'today': 5, 'found': 2, 'out': 6, 'about': 4, 'extreme': 5, 'setting': 9, 'e9': 5, 'squeezed': 2, 'files': 7, 'down': 6, 'another': 4, '15': 5, 'or': 2, 'so': 1, 'it': 6, 'is': 5, 'exactly': 3, 'quick': 1, "doesn't": 2, 'matter': 2, 'such': 4, 'cases': 1, 'securities': 9, 'exchange': 7, 'commission': 0, 'grad': 8, 'students': 5, 'get': 6, 'paid': 2, 'lot': 6, 'have': 1, 'solid': 4, 'job': 0, 'security': 5, 'ensured': 2, 'stress': 3, 'free': 9, 'environment': 4, 'healthy': 9, 'life': 3, 'balance': 3, 'still': 4, 'design': 0, 'proce

In [101]:
for dimensionality in [10,50,100,500,1000,2000,5000,10000]:
    index, collisions = hashwords(dimensionality)
    print(f"{len(collisions):2d}/{len(words):2d}={(len(collisions)*100)/len(words):4.1f}% collisions using {dimensionality:6d} unique codes")

10/105= 9.5% collisions using     10 unique codes
30/105=28.6% collisions using     50 unique codes
29/105=27.6% collisions using    100 unique codes
 7/105= 6.7% collisions using    500 unique codes
 4/105= 3.8% collisions using   1000 unique codes
 3/105= 2.9% collisions using   2000 unique codes
 1/105= 1.0% collisions using   5000 unique codes
 0/105= 0.0% collisions using  10000 unique codes


**Conclusion**: Using my typical hash function it takes a big dimensionality before we reduce collisions. In this case, there are only 96 unique words which means unique (perfect) hashing requires a dimensionality of only 96. The hashing trick requires thousands of hash buckets before we get 0 collisions. I suppose that if a simple hash function worked well, we would need wording embeddings. :)

## Using Spacy to tokenize, add meta data

Before, assigning (hopefully unique) integers to each word,It's a good idea to processed the text more heavily and to insert special tags to represent information about the text.  Humans innately see a stream of words and construct a parse tree in their head, such as this is the subject and this is the verb. In order to help a model understand the text, giving hints about proper nouns, verbs, and sentence structure could be very useful.

Spacy is great library for getting information about text. Check out [Spacy 101](https://spacy.io/usage/spacy-101)

In [84]:
import spacy
from spacy.lang.en import English
#!python -m spacy download en_core_web_sm  # requires restart of jupyter kernal afterwards

In [129]:
import re
def compress_whitespace(s): # collapse things like "\n   \t  " with " "
    return re.sub(r"(\s+)", ' ', s)

In [123]:
nlp = spacy.load("en_core_web_sm") # When I use plain English() it doesn't seem to give POS info
for sample in samples:    
    doc = nlp(compress_whitespace(sample))
    tokens = list(doc)
    print(tokens[:20])

[Tesla, Motors, has, nothing, to, do, with, this, tweet, ., On, those, rare, occasions, when, I, really, ,, really, need]
[Securities, and, exchange, commission, has, nothing, to, do, with, this, tweet, ., Do, grad, students, get, paid, a, lot, ?]
[A, design, process, hyperfocused, on, A, /, B, testing, can, result, in, dark, patterns, even, if, that, ’s, not, the]


### Add parts-of-speech meta data

Adding some tricks from [Jeremy / Sylvain's awesome book](https://github.com/fastai/fastbook/blob/master/10_nlp.ipynb) that indicate "start of stream" (`xxbos`) etc...  Use Spacy to add proper noun and verb tags.  Lowercase after Spacy has sniffed the text.

In [127]:
nlp = spacy.load("en_core_web_sm")
for sample in samples:
    doc = nlp(compress_whitespace(sample))
    tokens = ['xxbos']
    for token in doc:
        if token.pos_=='VERB':
            tokens.append('xxverb')
        if token.pos_=='PROPN':
            tokens.append('xxpropn')
        if token.dep_=='nsubj':           # is token subject of a sentence or phrase?
            tokens.append('xxsubj')
        tokens.append(str(token).lower())
    tokens.append('xxeos')
    print(' '.join(tokens[0:50]))

xxbos xxpropn tesla xxpropn xxsubj motors has nothing to do with this tweet . on those rare occasions when xxsubj i really , really xxverb need to xxverb reduce the size of a file xxsubj i xxverb use " xxpropn xz xxpropn -9 " . today xxsubj i xxverb found
xxbos xxpropn securities and xxpropn exchange xxpropn xxsubj commission has nothing to do with this tweet . do grad students get xxverb paid a lot ? no . but do xxsubj we at least have solid job security ? also , no . but are xxsubj we at least xxverb
xxbos a design xxsubj process xxverb hyperfocused on xxpropn a / b testing xxverb can xxverb result in dark patterns even if xxsubj that xxverb ’s not the intent . that xxverb ’s because most a / b tests are xxverb based on metrics xxsubj that are relevant to the


### Identify sentences and phrase subject

Add `xxbegin` tokens before start of each sentence.

In [128]:
from spacy.pipeline import Sentencizer

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(Sentencizer())
for sample in samples:
    doc = nlp(compress_whitespace(sample))
    tokens = ['xxbos']
    for token in doc:
        if token.pos_=='VERB':
            tokens.append('xxverb')
        if token.pos_=='PROPN':
            tokens.append('xxpropn')
        if token.dep_=='nsubj':           # is token subject of a sentence or phrase?
            tokens.append('xxsubj')
        if token.is_sent_start:           # beginning of sentence?
            tokens.append('xxbegin')
        tokens.append(str(token).lower())
    tokens.append('xxeos')
    print(' '.join(tokens[0:50]))

xxbos xxpropn xxbegin tesla xxpropn xxsubj motors has nothing to do with this tweet . xxbegin on those rare occasions when xxsubj i really , really xxverb need to xxverb reduce the size of a file xxsubj i xxverb use " xxpropn xz xxpropn -9 " . xxbegin today xxsubj
xxbos xxpropn xxbegin securities and xxpropn exchange xxpropn xxsubj commission has nothing to do with this tweet . xxbegin do grad students get xxverb paid a lot ? xxbegin no . xxbegin but do xxsubj we at least have solid job security ? xxbegin also , no . xxbegin but
xxbos xxbegin a design xxsubj process xxverb hyperfocused on xxpropn a / b testing xxverb can xxverb result in dark patterns even if xxsubj that xxverb ’s not the intent . xxbegin that xxverb ’s because most a / b tests are xxverb based on metrics xxsubj that are relevant


Once we have everything tokenized properly, we can then assign a unique number to the words and meta-tags.